<a href="https://colab.research.google.com/github/fatihdzaki01/Projek_PDS_RegresiLinier/blob/DarellX/projek_pds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. membuat model berdasarkan korelasi terbesar, Multikol terkecil


## 1.1 IMPORT LIBRARY AND LOAD DATASET

In [ ]:
#Import Library yang Dibutuhkan

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from folium import Choropleth
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro
import statsmodels.api as sm


# tampilkan semua kolom
pd.set_option('display.max_columns', None)

In [ ]:
# --- Import Data ---
df = pd.read_csv("air_quality.csv")

# lihat 5 baris pertama
df.head()


In [ ]:
#lakukan pengarsipan
df_raw = df.copy()

## 1.2 DATA PREPROCESSING

### 1.2.1 Eksplorasi awal

In [ ]:
#Eksplorasi Awal
df.info()
df.describe()

### 1.2.2 Penyesuaian Tipe Data

In [ ]:
# lihat tipe data yang sekarang
df.dtypes

In [ ]:
# ubah beberapa kolom menjadi numerik
cols_num = ['aqi', 'so2', 'co', 'o3', 'o3_8hr', 'pm10', 'pm2.5', 'no2',
             'nox', 'no', 'windspeed', 'winddirec', 'co_8hr',
             'pm2.5_avg', 'pm10_avg', 'so2_avg']

for c in cols_num:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

In [ ]:
# ubah tipedata kolom 'date' menjadi date
df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')

In [ ]:
# cek kembali tipedata , untuk memastikan sudah sesuai
df.dtypes

### 1.2.3 Cek Missing Value

In [ ]:
# cek missing value
print(df.isnull().sum())

lakukan penghapusan row dimana longitude, latitude dan siteid nya NULL, karena tidak bisa dilakukan visualisasi MAP

In [ ]:
# hapus row dengan kolom longitude, latitude dan siteid NULL, karena tidak bisa dibuat visualisasi map
id_cols = ['longitude', 'latitude', 'siteid']
for c in id_cols:
    if c in df.columns:
        df = df.dropna(subset=[c])

In [ ]:
# persentase missing value
(df.isnull().sum() / len(df)) * 100

karena kolom 'pollutant' dan 'unit' memiliki persentase missing value cukup besar (> 50%) maka kolom tersebut kami hapus

In [ ]:
# hapus kolom pollutant dan unit
df.drop(columns=['unit', 'pollutant'], inplace = True)

untuk menentukan apakah missing value di isi menggunakan median atau mean, wajib melihat outlier dan distribusinya. jika berdistribusi normal, maka sebaiknya menggunakan mean, jika tidak berdistribusi normal dan punya banyak outlier maka menggunakan median

### 1.2.4 cek outlier

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Pilih kolom numerik
numeric_cols = df.select_dtypes(include=['float64']).columns

n_cols = 3  # jumlah kolom di grid
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

# Buat figure
plt.figure(figsize=(n_cols * 5, n_rows * 4))

# Loop tiap kolom dan plot boxplot-nya
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    plt.boxplot(df[col].dropna(), vert=True)
    plt.title(col)
    plt.tight_layout()

plt.show()


karena hampir semua kolom memilki nilai outlier, oleh karena itu pengisian missing value menggunakan nilai median

In [ ]:
# Isi missing value pada kolom numerik dgn median karena lebih tahan terhadap outlier dibanding mean
for c in cols_num:
    if c in df.columns:
        median_value = df[c].median()
        df[c] = df[c].fillna(median_value)

In [ ]:
# menangani kolom status
# cek nilai unique
df['status'].unique()

In [ ]:
df.groupby('status')['aqi'].agg(['min', 'max']).reset_index()

In [ ]:
# fungsi untuk menentukan status berdasarkan nilai AQI
def categorize_aqi(aqi):
    if pd.isna(aqi):
        return None
    elif 0 <= aqi <= 50:
        return 'Good'
    elif 51 <= aqi <= 100:
        return 'Moderate'
    elif 101 <= aqi <= 150:
        return 'Unhealthy for Sensitive Groups'
    elif 151 <= aqi <= 200:
        return 'Unhealthy'
    elif 201 <= aqi <= 300:
        return 'Very Unhealthy'
    elif aqi >= 301:
        return 'Hazardous'
    else:
        return 'Unknown'

In [ ]:
df['status'] = df.apply(
    lambda row: categorize_aqi(row['aqi']) if pd.isna(row['status']) else row['status'],
    axis=1
)

In [ ]:
# --- Cek hasil akhir ---
print("Missing values setelah pembersihan:\n", df.isnull().sum())

In [ ]:
#lakukan pengarsipan untuk missing value
df_missingvalue = df.copy()

### 1.2.5 Cek Duplikasi data

In [ ]:
#cek duplikat
df.duplicated().any()

In [ ]:
# melihat duplikasi

dupes = df[df.duplicated(keep=False)]
dupes.sort_values(by=list(df.columns)).head(20)

In [ ]:
#hapus duplikat
df.drop_duplicates(inplace=True)

In [ ]:
df_duplikasi = df.copy()

### 1.2.6 Cek Distribusi

cara paling mudah dan simple melihat distribusi ialah dengan menggunakan histogram.

In [ ]:
# Pilih hanya kolom numerik
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

#jumlah kolom grid
n_cols = 3
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

# Buat figure
plt.figure(figsize=(n_cols * 3, n_rows * 2))

# Loop untuk setiap kolom
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    plt.hist(df[col].dropna(), bins=30, edgecolor='black')
    plt.title(col)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()

plt.show()

lakukan transformasi log untuk kolom yang skeewness right (mayoritas kolom spt itu)

In [ ]:
import numpy as np

# daftar kolom yang mau di-log transform
right_skewed_cols = [
    'aqi', 'o3_8hr', 'nox', 'no2', 'no',
    'windspeed', 'pm2.5_avg', 'pm10_avg', 'so2_avg'
]

LOG_THRESHOLD_MAX = 10
LOG_THRESHOLD_MEAN = 2

for col in right_skewed_cols:
    if col not in df.columns:
        print(f"⚠️ Kolom '{col}' tidak ditemukan di DataFrame, skip.")
        continue

    # isi nilai kosong dulu biar gak NaN setelah log
    df[col] = df[col].fillna(0)

    # buang nilai negatif (ganti jadi 0 biar aman di log1p)
    df[col] = df[col].clip(lower=0)

    # ambil statistik dasar
    max_val = df[col].max()
    mean_val = df[col].mean()

    # deteksi apakah sudah di-log sebelumnya
    if max_val < LOG_THRESHOLD_MAX and mean_val < LOG_THRESHOLD_MEAN:
        print(f"⏭️  Kolom '{col}' kemungkinan SUDAH di-log sebelumnya — dilewati.")
        continue

    # lakukan log transform aman
    df[col] = np.log1p(df[col])

    # pastikan gak ada inf/NaN setelah log
    df[col].replace([np.inf, -np.inf], np.nan, inplace=True)
    df[col].fillna(df[col].median(), inplace=True)

    print(f"✅ Log-transform aman diterapkan ke kolom: '{col}' (max={max_val:.2f}, mean={mean_val:.2f})")


lihat histogram pasca di normalisasi

In [ ]:
right_skewed_cols = [
    'aqi', 'o3_8hr', 'nox', 'no2', 'no',
    'windspeed', 'pm2.5_avg', 'pm10_avg', 'so2_avg'
]

# buat grid histogram
n_cols = 3  # jumlah kolom plot per baris
n_rows = int(np.ceil(len(right_skewed_cols) / n_cols))

plt.figure(figsize=(15, 10))
for i, col in enumerate(right_skewed_cols, 1):
    if col in df.columns:
        plt.subplot(n_rows, n_cols, i)
        plt.hist(df[col].dropna(), bins=30, edgecolor='black')
        plt.title(col, fontsize=10)
        plt.xlabel("Value")
        plt.ylabel("Frequency")
    else:
        print(f"⚠️ Kolom '{col}' tidak ditemukan di DataFrame — skip.")

plt.tight_layout()
plt.show()

In [ ]:
#lakukan pengarsipan
df_transform = df.copy()

### 1.2.7 cek korelasi

In [ ]:
corr_matrix = df[["so2","co","o3","o3_8hr","pm10","pm2.5","no2","nox","no",
        "windspeed","winddirec","co_8hr","pm2.5_avg","pm10_avg","so2_avg", "aqi"]].corr()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Heatmap Korelasi Antar Variabel")
plt.show()

dari Map Korelasi diatas, yang akan digunakan dalam membuat model adalah co, so2_avg, windspeed, o3 dan pm2.5_avg

### 1.2.8 Cek Multikolinieritas

In [ ]:
cols_to_check = ["co", "so2_avg", "windspeed", "o3", "pm2.5_avg"]
X = df[cols_to_check].copy()
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [ ]:
print(vif_data.sort_values(by="VIF", ascending=False))

In [ ]:
#buat map korelasi hanya untuk kolom yang akan kita gunakan dalam model
corr_matrix = df[["co","o3","winddirec","so2_avg", "pm2.5_avg", "aqi"]].corr()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Heatmap Korelasi Antar Variabel")
plt.show()

## 1.3 Membuat Model

### 1.3.1 definisikan target dan fitur

In [ ]:
df = pd.read_csv("data_sampemodel.csv")

karena ini merupakan data timeseries, maka akan kita cek terlebih dahulu tahunnya

In [ ]:
#convert data ke datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df['date'].dtype

In [ ]:
#lihat range data
df["date"].min(), df["date"].max()

karena range data nya dari 15 juli 2019 - 31 agustus 2024, maka untuk split data train adalah tahun 2020-2022 dan data training tahun 2023

In [ ]:
# kita buat kolom khusus untuk tahun
df["year"] = df["date"].dt.year

In [ ]:
train_df = df[(df["year"] >= 2020) & (df["year"] <= 2022)]
test_df  = df[df["year"] == 2023]

In [ ]:
X_train = train_df[["co", "so2_avg", "windspeed", "o3", "pm2.5_avg"]]
y_train = train_df["aqi"]

X_test = test_df[["co", "so2_avg", "windspeed", "o3", "pm2.5_avg"]]
y_test = test_df["aqi"]

In [ ]:
import statsmodels.api as sm

X_train = sm.add_constant(X_train)
X_test  = sm.add_constant(X_test)

### 1.3.2 print modelnya

In [ ]:
X_train.isna().sum()

In [ ]:
# Hapus semua baris yang punya NaN di X_train
X_train = X_train.dropna()

# Samain y_train biar indexnya match dengan X_train
y_train = y_train.loc[X_train.index]

In [ ]:
# model OLS (Ordinary Least Squares)
model = sm.OLS(y_train, X_train).fit()
print(model.summary())

prediksi pakai data test (data tahun 2023)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

## 1.4 Uji Asumsi Klasik

### 1.4.1  Uji Homoskedastisitas

In [ ]:
#uji homoskedastisitas
from statsmodels.stats.diagnostic import het_breuschpagan

# model = sm.OLS(y, X).fit()
# Ambil residual dan variabel independen
residuals = model.resid
exog = model.model.exog

# Lakukan uji Breusch-Pagan
bp_test = het_breuschpagan(residuals, exog)

# Simpan hasilnya
labels = ['Lagrange multiplier statistic', 'p-value',
          'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))


In [ ]:
#visualisasi homoskedastisitas
#sekalian uji linearitas
import matplotlib.pyplot as plt
import seaborn as sns

# Hitung residual dan prediksi
residuals = model.resid
fitted = model.fittedvalues

plt.figure(figsize=(8,5))
sns.scatterplot(x=fitted, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

#heteroskedastisitas ringan
#uji linearitas masih cukup terpenuhi

### 1.4.2 Uji Multikolinieritas

In [ ]:
#uji multikolinearitas dari data yang dipake di model
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# Hitung VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

#udah bagus VIF nya

### 1.4.3 Uji Normalitas
disini kita menggunakan test Jarque Bera Test karena data yang digunakan ialah data Time Series dan Jarque Bera Test lebih cocok untuk data Time series

In [ ]:
# statistik jarque bera test
from statsmodels.stats.stattools import jarque_bera
residual = model.resid
jb_stat, jb_pvalue, skew, kurtosis = jarque_bera(residual)

print("\n=== UJI NORMALITAS (JARQUE-BERA TEST) ===")
print(f"Jarque-Bera Statistic : {jb_stat:.4f}")
print(f"P-Value               : {jb_pvalue:.4f}")
print(f"Skewness              : {skew:.4f}")
print(f"Kurtosis              : {kurtosis:.4f}")

# Interpretasi otomatis
if jb_pvalue > 0.05:
    print("✅ Residual berdistribusi normal (p > 0.05)")
else:
    print("❌ Residual tidak normal (p < 0.05)")

In [ ]:
#visual
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

# Histogram residual + KDE
sns.histplot(residual, kde=True)
plt.title("Distribusi Residual")
plt.show()

# Q-Q plot (normal probability plot)
stats.probplot(residual, dist="norm", plot=plt)
plt.title("Q-Q Plot Residual")
plt.show()


ada sedikit keanehan pada distribusi residualnya, disini nilai nya berkumpul pada rentang -2 hingga 2, salah satu penyebabnya ialah range data yang berbeda jauh

In [ ]:
# Lihat range (max - min) dan standar deviasi
X_train.describe().T[["min", "max", "mean", "std"]]

In [ ]:
# disini ada keanehan yaitu data min untuk co dan o3 sebesar -999.0, oleh karena itu kita ubah -999.0 menjadi NAN. dan diubah menjadi median

import numpy as np

cols_to_fix = ["co", "o3"]  # ganti sesuai kolom yang bermasalah

# Ganti -999 jadi NaN di semua kolom
X_train[cols_to_fix] = X_train[cols_to_fix].replace(-999, np.nan)
X_test[cols_to_fix] = X_test[cols_to_fix].replace(-999, np.nan)

# Isi NaN dengan median (pakai median dari TRAIN)
for col in cols_to_fix:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)


In [ ]:
#kita test lagi dengan jerque bera test
model = sm.OLS(y_train, X_train).fit()
residual = model.resid
jb_stat, jb_pvalue, skew, kurtosis = jarque_bera(residual)

print("\n=== UJI NORMALITAS (JARQUE-BERA TEST) ===")
print(f"Jarque-Bera Statistic : {jb_stat:.4f}")
print(f"P-Value               : {jb_pvalue:.4f}")
print(f"Skewness              : {skew:.4f}")
print(f"Kurtosis              : {kurtosis:.4f}")

# Interpretasi otomatis
if jb_pvalue > 0.05:
    print("✅ Residual berdistribusi normal (p > 0.05)")
else:
    print("❌ Residual tidak normal (p < 0.05)")

In [ ]:
(X_train == 0).sum() / len(X_train) * 100


In [ ]:
# Transform sqrt langsung pada kolom asli
X_train["so2_sqrt"] = np.sqrt(X_train["so2_avg"])

# Tambahkan fitur indikator zero
X_train["so2_zero"] = (X_train["so2_avg"] == 0).astype(int)

In [ ]:
X_train = X_train.drop(columns=["so2_avg"])

In [ ]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary())

In [ ]:
#visual
sns.histplot(model.resid, bins=50, kde=True)
plt.title("Distribusi Residual")
plt.show()

In [ ]:
res = model.resid

In [ ]:
lower = res.quantile(0.01)
upper = res.quantile(0.99)

In [ ]:
mask = (res >= lower) & (res <= upper)

X_train_trim = X_train[mask]
y_train_trim = y_train[mask]

In [ ]:
model_trim = sm.OLS(y_train_trim, X_train_trim).fit()

In [ ]:
sns.histplot(model_trim.resid, bins=50, kde=True)
plt.title("Residual setelah trimming outlier")
plt.show()

Udah aman

# 2. MEMBUAT PERBANDINGAN ANTARA MODEL DENGAN VARIABEL LENGKAP DENGAN VARIABEL PILIHAN

## 2.1 pemilihan variabel, split data, deklarasi x dan y

In [ ]:
#perbandingan model dengan variabel lengkap
import pandas as pd
import statsmodels.api as sm

# 1. Buat kolom tahun dari date
df["year"] = df["date"].dt.year

# 2. Split train-test berdasarkan tahun
train_df_v = df[(df["year"] >= 2020) & (df["year"] <= 2022)]
test_df_v  = df[df["year"] == 2023]

# 3. Daftar kolom yang TIDAK boleh masuk sebagai predictor
exclude_cols = [
    "date", "sitename", "county", "aqi", "pollutant", "status",
    "unit", "longitude", "latitude", "siteid", "year"
]

# 4. Pilih semua kolom numerik kecuali yang dikecualikan
features = [
    col for col in train_df_v.select_dtypes(include=["float64", "int64"]).columns
    if col not in exclude_cols
]

print("Fitur yang dipakai sebagai X:")
print(features)

# 5. Buat X dan y
X_train_v = train_df_v[features]
y_train_v = train_df_v["aqi"]

X_test_v  = test_df_v[features]
y_test_v  = test_df_v["aqi"]

# 6. Tambahkan konstanta
X_train_v = sm.add_constant(X_train_v)
X_test_v  = sm.add_constant(X_test_v)


## 2.1 MEMBUAT MODEL

### 2.1.1 memilih model ols dan print summary

In [ ]:
# model OLS (Ordinary Least Squares)
model_v = sm.OLS(y_train_v, X_train_v).fit()
print(model_v.summary())

### 2.1.2 deklarasi y_predict

In [ ]:
y_pred_v = model_v.predict(X_test_v)

### 2.1.3 Evaluasi Model

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test_v, y_pred_v)
rmse = mse**0.5
r2 = r2_score(y_test_v, y_pred_v)

print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)

**PERBANDINGAN**

R-Squared: Model lengkap menjelaskan persentase variasi yang lebih besar pada variabel dependen (aqi)


F-statistic: kedua model secara keseluruhan sangat signifikan (Prob (F-statistic) = 0.00). Nilai F yang lebih besar pada Model dengan 5 variabel (meskipun memiliki $R^2$ lebih kecil) menunjukkan bahwa variabelnya secara kolektif menjelaskan variasi yang jauh lebih besar relatif terhadap jumlah variabelnya yang lebih kecil.


Condition Number: Kekhawatiran Utama. Angka di atas 30 menunjukkan multikolinearitas yang kuat. Model lengkap memiliki 4110, menunjukkan masalah multikolinearitas yang sangat parah, yang dapat membuat koefisien menjadi tidak stabil dan sulit diinterpretasikan. Model dengan 5 variabel juga memiliki Cond. No. yang tinggi (179) tetapi jauh lebih rendah daripada Model 1, mengindikasikan bahwa pengurangan variabel telah secara signifikan mengurangi tingkat multikolinearitas, meskipun masih ada.


**KESIMPULAN**
1. Daya Penjelas (Fit): Model lengkap lebih unggul dalam hal $R^2$ dan Adjusted $R^2$ (0.795 vs 0.764) dan memiliki nilai AIC/BIC yang jauh lebih rendah, menunjukkan bahwa model tersebut secara statistik merupakan model yang lebih baik untuk menjelaskan variasi variabel dependen ($aqi$).
2. Masalah Kualitas Model (Multikolinearitas): Model lengkap mengalami masalah yang jauh lebih parah dengan multikolinearitas (Cond. No. $4110$) dibandingkan Model dengan 5 variabel (Cond. No. 179). Hal ini ditunjukkan oleh koefisien $co$ yang berubah tanda secara drastis antara kedua model, yang merupakan tanda klasik dari ketidakstabilan koefisien akibat multikolinearitas yang tinggi.
3. Keterbatasan Umum: Kedua model menunjukkan bukti kuat adanya autokorelasi (Durbin-Watson < 2) dan residual yang tidak berdistribusi normal (Prob(Omnibus) = 0.000).



-------------------

MODEL OLS & MLE COMPARISON

-------------------

Komparasi 16 Parameter

In [ ]:
# model OLS (Ordinary Least Squares)
model_v = sm.OLS(y_train_v, X_train_v).fit()
print(model_v.summary())

In [ ]:
model_mle_v = sm.GLM(y_train_v, X_train_v, family=sm.families.Gaussian()).fit()
print(model_mle_v.summary())


In [ ]:
comparison = pd.DataFrame({
    "Model": ["OLS", "MLE"],
    "Parameters": [len(model_v.params), len(model_mle_v.params)],
    "AIC": [model_v.aic, model_mle_v.aic],
    "BIC": [model_v.bic, model_mle_v.bic],
    "Log-Likelihood": [model_v.llf, model_mle_v.llf]
})
print(comparison)


Komparasi 7 Parameter ( PM2.5 )

In [ ]:
# model OLS (Ordinary Least Squares)
model = sm.OLS(y_train, X_train).fit()
print(model.summary())

In [ ]:
model_mle_6Parameter = sm.GLM(y_train, X_train, family=sm.families.Gaussian()).fit()
print(model_mle_6Parameter.summary())

In [ ]:
comparison = pd.DataFrame({
    "Model": ["OLS", "MLE"],
    "Parameters": [len(model.params), len(model_mle_6Parameter.params)],
    "AIC": [model.aic, model_mle_6Parameter.aic],
    "BIC": [model.bic, model_mle_6Parameter.bic],
    "Log-Likelihood": [model.llf, model_mle_6Parameter.llf]
})
print(comparison)


Baik OLS maupun MLE menghasilkan AIC dan log-likelihood yang identik. Ini menunjukkan bahwa kedua metode memberikan kualitas prediksi dan kompleksitas model yang sama. Dengan demikian, tidak terdapat perbedaan performa antara OLS dan MLE pada dataset ini.

-----------------------------------------------------

PM2.5 -> PM10 7 Parameter

-----------------------------------------------------

In [ ]:
# --- Fitur PM10 ---
fitur_pm10 = ["co", "so2_avg", "windspeed", "o3", "pm10_avg"]

# --- Train set ---
X_train_pm10 = train_df[fitur_pm10].copy()
y_train_pm10 = train_df["aqi"]

# Transformasi SO2
X_train_pm10["so2_sqrt"] = np.sqrt(X_train_pm10["so2_avg"])
X_train_pm10["so2_zero"] = (X_train_pm10["so2_avg"] == 0).astype(int)
X_train_pm10 = X_train_pm10.drop(columns=["so2_avg"])

# --- Test set ---
X_test_pm10 = test_df[fitur_pm10].copy()
y_test_pm10 = test_df["aqi"]

# Transformasi SO2 sama seperti train
X_test_pm10["so2_sqrt"] = np.sqrt(X_test_pm10["so2_avg"])
X_test_pm10["so2_zero"] = (X_test_pm10["so2_avg"] == 0).astype(int)
X_test_pm10 = X_test_pm10.drop(columns=["so2_avg"])

In [ ]:
X_train_pm10 = sm.add_constant(X_train_pm10)
X_test_pm10  = sm.add_constant(X_test_pm10)

In [ ]:
# OLS
model_ols_pm10 = sm.OLS(y_train_pm10, X_train_pm10).fit()
print(model_ols_pm10.summary())


In [ ]:
# MLE Gaussian
model_mle_pm10 = sm.GLM(y_train_pm10, X_train_pm10, family=sm.families.Gaussian()).fit()
print(model_mle_pm10.summary())

In [ ]:
comparison_pm10 = pd.DataFrame({
    "Model": ["OLS", "MLE"],
    "Parameters": [len(model_ols_pm10.params), len(model_mle_pm10.params)],
    "AIC": [model_ols_pm10.aic, model_mle_pm10.aic],
    "BIC": [model_ols_pm10.bic, model_mle_pm10.bic],
    "Log-Likelihood": [model_ols_pm10.llf, model_mle_pm10.llf]
})

print(comparison_pm10)

# 3. MEMBUAT MODEL TANPA MELAKUKAN TRANSFORMASI

##3. 1 Deklarasi variabel

In [ ]:
df_no_log = df_duplikasi.copy()

In [ ]:
df_no_log["date"] = pd.to_datetime(df_no_log["date"], errors="coerce")

# bikin kolom tahun
df_no_log["year"] = df_no_log["date"].dt.year

## 3.2 melakukan split data training dan testing

In [ ]:
train_df = df_no_log[(df_no_log["year"] >= 2020) & (df_no_log["year"] <= 2022)]
test_df  = df_no_log[df_no_log["year"] == 2023]

print("Jumlah data train :", len(train_df))
print("Jumlah data test  :", len(test_df))

In [ ]:
fitur = ["co", "so2_avg", "windspeed", "o3", "pm2.5_avg"]

X_train = train_df[fitur]
y_train = train_df["aqi"]

X_test = test_df[fitur]
y_test = test_df["aqi"]

In [ ]:
X_train = sm.add_constant(X_train)
X_test  = sm.add_constant(X_test)

## 3.3 Deklarasi model

In [ ]:
model_no_log = sm.OLS(y_train, X_train).fit()

print(model_no_log.summary())

### 3.3.1 mencari y preddict

In [ ]:
y_pred = model_raw.predict(X_test)

### 3.3.2 evaluasi model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

print("===== Evaluasi Model Tanpa Transformasi =====")
print("MSE :", mse)
print("RMSE:", rmse)
print("R²  :", r2)

3.3.3 melihat range variabel y untuk melihat apakah nilai MSE dan RMSE buruk atau baik

Untuk mengetahui baik buruknya MSE maka harus di cari dulu nilai MIN dan MAX dari AQI

In [ ]:
print("===== MIN–MAX AQI SELURUH DATA =====")
print("Min AQI :", df_no_log['aqi'].min())
print("Max AQI :", df_no_log['aqi'].max())

**KESIMPULAN**

R-squared 0.913 (train) dan R² test 0.901, membuktikan model mampu menjelaskan sebagian besar variasi AQI. Semua variabel signifikan secara statistik, dengan PM2.5 sebagai prediktor paling dominan. RMSE hanya 8.10, jauh lebih kecil dari rentang AQI (−1 sampai 467), sehingga error model relatif sangat kecil. Meskipun bhasilnya bagus, distribusi residual sangat skewed, sehingga model mungkin diuntungkan jika dilakukan transformasi log pada variabel yang right-skewed.

# STOPPPP